In [1]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 9.2 MB/s eta 0:00:009.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for transformers<5.0.0,>=4.6.0 from https://files.pythonhosted.org/packages/98/46/f6a79f944d5c7763a9bc13b2aa6ac72daf43a6551f5fb03bccf0a9c2fec1/transformers-4.33.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 8.3 MB/s eta 0:00:00
  Using cached torch-2.0.1-cp310-none-macosx_11_0_arm64.whl (55.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.1 MB/s eta 0:00:0031m10.8 MB/s eta 0:00:01
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/68/43/5f371523089871c0190800d64a9483f86ae68cd8360a86f529596b3a0496/scikit_learn-1.3.1-cp310-cp310-macosx_12_0_arm64.whl.metadata
  Using cached scikit_learn-1.3.1-cp310-cp310-macosx_12_0_arm64.whl.metadata 

Using cached scipy-1.11.3-cp310-cp310-macosx_12_0_arm64.whl (29.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.9/406.9 kB 9.3 MB/s eta 0:00:000:00:01
Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 11.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=69c093f15a9cfd80705076f053f47e616b800069c123a6de9a484c052973b7b5
  Stored in directory: /Users/Vlahonator/Library/Caches/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [5]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [6]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [7]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


## Step 1 : Create source embeddings for the text column

In [8]:
from sentence_transformers import SentenceTransformer

/Users/Vlahonator/anaconda3/envs/nlp_envir/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [10]:
vectors.shape

(8, 768)

In [19]:
vectors

array([[-0.00247378,  0.03626734, -0.05290457, ..., -0.0915236 ,
        -0.03970009, -0.04330485],
       [-0.03357262,  0.00980523, -0.03250129, ..., -0.05165472,
         0.02245885, -0.03156187],
       [-0.01865326, -0.04051305, -0.01235391, ...,  0.00610579,
        -0.0717965 ,  0.02773846],
       ...,
       [-0.00066462,  0.04252121, -0.05645506, ...,  0.01315471,
        -0.03183562, -0.04357661],
       [-0.03317155,  0.03252463, -0.02484844, ...,  0.01174419,
         0.05747125,  0.00571023],
       [-0.00166398,  0.00413837, -0.04597078, ...,  0.02008531,
         0.05656243, -0.00161598]], dtype=float32)

In [11]:
dim = vectors.shape[1]
dim

768

## Step 2 : Build a FAISS Index for vectors

In [12]:
import faiss

index = faiss.IndexFlatL2(dim)

## Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [13]:
index.add(vectors)

In [14]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x297ec5e90> >

## Step 4 : Encode search text using same encorder and normalize the output vector

In [15]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [16]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [17]:
# faiss.normalize_L2(svec)

## Step 5: Search for similar vector in the FAISS index created

In [20]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844838, 1.4039094]], dtype=float32)

In [21]:
I

array([[3, 2]])

In [22]:
I.tolist()

[[3, 2]]

In [23]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [24]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [25]:
search_query

'I want to buy a polo t-shirt'